In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

In [2]:
data = pd.read_csv('../Dataset/combined_final_dataset.csv')

le = LabelEncoder()
le.fit(data["main_category"])
le_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
z = data.pop("date")
del(z)
m = data.pop("category")
del(m)
n = data.pop("Title")
del(n)
data["main_category"] = LabelEncoder().fit_transform(data["main_category"])
y = data.pop('main_category')
print(le_mapping)

{'Code': 0, 'Medical': 1, 'News': 2, 'Research Paper': 3}


In [3]:
print(data.head())
print(data.columns)
y.value_counts()

                                         description
0  health expert said early predict whether deman...
1  subdued passenger crew fled back aircraft conf...
2                         dog understand could eaten
3  accidentally put toothpaste toothbrush screame...
4  amy cooper accused investment firm franklin te...
Index(['description'], dtype='object')


1    349
2    324
3     61
0     33
Name: main_category, dtype: int64

In [4]:
import numpy
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)


#convert to numpy array
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(613, 1)
(154, 1)
(613,)
(154,)


In [46]:
vocab_size = 20000
embedding_dim = 32
max_length = 200
trunc_type='post'
padding_type='pre'

tokenizer = Tokenizer(num_words = vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

In [47]:
x_train.shape
x_train_sentences = x_train['description'].tolist()
x_test_sentences = x_test['description'].tolist()

In [48]:
sequences = tokenizer.texts_to_sequences(x_train_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_sequences = tokenizer.texts_to_sequences(x_test_sentences)
test_padded = pad_sequences(test_sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [49]:
print(padded.shape)

(613, 200)


In [50]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [51]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
#



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 32)           640000    
                                                                 
 flatten_4 (Flatten)         (None, 6400)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               819328    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 4)                 516       
                                                                 
Total params: 1459844 (5.57 MB)
Trainable params: 1459844 (5.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
model.fit(padded, y_train, epochs=30, validation_data=(test_padded, y_test))

Epoch 1/30


20/20 [==============================] - 1s 33ms/step - loss: 0.7086 - accuracy: 0.8010 - val_loss: 0.4768 - val_accuracy: 0.8377
Epoch 2/30
20/20 [==============================] - 0s 22ms/step - loss: 0.4200 - accuracy: 0.8744 - val_loss: 0.4008 - val_accuracy: 0.8442
Epoch 3/30
20/20 [==============================] - 0s 24ms/step - loss: 0.3781 - accuracy: 0.8825 - val_loss: 0.3823 - val_accuracy: 0.8636
Epoch 4/30
20/20 [==============================] - 1s 26ms/step - loss: 0.3538 - accuracy: 0.8825 - val_loss: 0.3680 - val_accuracy: 0.8506
Epoch 5/30
20/20 [==============================] - 0s 24ms/step - loss: 0.3570 - accuracy: 0.8809 - val_loss: 0.3674 - val_accuracy: 0.8636
Epoch 6/30
20/20 [==============================] - 1s 25ms/step - loss: 0.3464 - accuracy: 0.8874 - val_loss: 0.3866 - val_accuracy: 0.8636
Epoch 7/30
20/20 [==============================] - 0s 24ms/step - loss: 0.3554 - accuracy: 0.8825 - val_loss: 0.3838 - val_accuracy: 0.8506
Epoch 8/30
20/20 [======

In [54]:
new_description = ["The conflict has led to the death of over 700 Israelis and thousands of others have been injured. Meanwhile, the death toll in Gaza has also risen to 493, suggest reports. US President Joe Biden "]
seq = tokenizer.texts_to_sequences(new_description)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
print(pred[0])
#convert num back to label
print(list(le_mapping.keys())[list(le_mapping.values()).index(np.argmax(pred))])



1/1 [==============================] - 0s 42ms/step


[0.14174145 0.6765909  0.00516331 0.17650425]
Medical


In [13]:
from sklearn.utils import class_weight
import numpy as np

# Assuming 'y_train' contains your integer-encoded class labels
class_labels = np.unique(y_train)

# Calculate class weights
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = class_labels, y = y_train)

# Create a dictionary mapping class indices to their respective weights
class_weight_dict = dict(zip(class_labels, class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {0: 5.675925925925926, 1: 0.5434397163120568, 2: 0.5916988416988417, 3: 3.4055555555555554}


AssertionError: Torch not compiled with CUDA enabled